# 作業工程計画

0. ライブラリを読み込む
1. 過去データを読み込む
2. 数学だけ抜き出す
3.　最終ペースに学年による有意差はあるか？
4.　分野で分けた場合に、3はどうなる？
5.　分野別最終ペースを多い順に並べて順序変数を振る
6.　5を分野に代わる特徴料として使用する

## ライブラリ

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
import re 

"""
機械学習ライブラリの準備
"""

from sklearn.model_selection import cross_val_score
# from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.tree import DecisionTreeRegressor #決定木
import lightgbm as lgb #lightGBM

from sklearn import tree
import graphviz

#ライブラリの読み込み
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import  KFold
import warnings
warnings.filterwarnings('ignore')

### 村上さん pandas_tool

In [2]:
"""
村上さんtoolbox
"""
# pandas 基礎集計クラス
import numpy as np
import pandas as pd
import copy
import seaborn as sns
from itertools import combinations

#http://qiita.com/tanemaki/items/2ed05e258ef4c9e6caac

# Jupyterで表示するためには、最初に以下を実行すること
%matplotlib inline

# Static Classで設計する
class pandas_tool:
    
    # All in one チェック（Jupyterのみ）
    def all_basic_summary_jupyter(df):
        print("■ 型の確認")
        display(pandas_tool.type(df))
        print("■ 数値型の確認")
        display(pandas_tool.summary(df)[0])
        print("■ カテゴリ型の確認")
        cate_var_data = list(df.select_dtypes(include=['object']).columns)
        ret = pandas_tool.freq(df,cate_var_data)
        for d in ret:
            display(pd.DataFrame(d))
            print("---------------")
        print("■ 欠損の確認")
        display(pandas_tool.check_missing(df))
    
    # 相関関係可視化（Jupyterのみ）
    def all_value_relation_visualize(df):
        #sns.set_context("poster", 1.2, {"lines.linewidth": 3})
        sns.pairplot(df,size=5)
    
    # カテゴリ変数でのヒートマップ（Jupyterのみ）
    def make_heatmap(df,x,y,value):
        target_df = df.pivot_table(index=x,values=value,columns=y)
        sns.heatmap(target_df, annot=True, fmt='1.1f', cmap='Blues')
    
    # 散布図（Jupyterのみ）
    def make_scatter_chart(df,x,y):
        #sns.jointplot(x=x, y=y, data=df, kind="hex")
        sns.jointplot(x=x, y=y, data=df)
    
    # 組み合わせでヒートマップを作成（Jupyterのみ）
    def all_make_heatmap(df,var_list,value):
        col_num = 2
        var_list_set = list(combinations(var_list,2))
        
        fig, axes = plt.subplots(int(len(var_list_set)/col_num)+1, col_num, figsize=(18,3+6.5*int(len(var_list_set)/col_num)))
        
        for i,target in enumerate(var_list_set):
            target_df = df.pivot_table(index=target[0],values=value,columns=target[1])
            sns.heatmap(target_df, annot=True, fmt='1.1f', cmap='Blues', ax=axes[int(i/col_num), i%col_num])
            
        plt.tight_layout()
    
    # 数値集計
    def summary(df,view=False):
        ret=df.describe()
        mis_ret=df.isnull().sum()
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print("・統計量")
            print(ret)
            print("・欠損値")
            print(mis_ret)
            pd.set_option("display.max_columns",param)
        return ret,mis_ret
    
    # 型チェック
    def type(df,view=False):
        ret = df.dtypes
        if view:
            param=pd.get_option("display.max_rows")
            pd.set_option("display.max_rows",1000)
            print(ret)
            pd.set_option("display.max_rows",param)
        return ret
    
    # 欠損チェック
    def check_missing(df,view=False):
        not_null_df=df.notnull()
        ret=pd.DataFrame()
        for name in not_null_df.columns:
            tmp_df=not_null_df[name].value_counts()
            tmp_df.name=name
            ret = pd.concat([ret,tmp_df],axis=1)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 欠損値のオブザベーションを抽出
    def get_miss_data(df,column,view=False):
        ret=df[df[column].isnull()]
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        return ret
    
    # 欠損値を中央値で補完
    def fill_miss_med(df,var_name):
        var=df[var_name].median()
        df[var_name].fillna(var,inplace=True)
        return df
    
    # 欠損値を0で補完
    def fill_miss_zero(df,var_name):
        df[var_name].fillna(0,inplace=True)
        return df
    
    # 特定の値を欠損とみなす
    def apply_miss_value(df,var_name,value):
        df[var_name]=df[var_name].replace(value,np.nan)
        return df
    
    # 重複チェック
    def check_dup(df,columns,view=False):
        ret=pd.DataFrame()
        for name in columns:
            dup_cnt=df[name].duplicated().sum()
            tmp_df = pd.DataFrame({'var_name':[name],'dup_cnt':[dup_cnt]})
            ret = pd.concat([ret,tmp_df],axis=0,ignore_index= True)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 組み合わせ重複チェック
    def check_dup_comb(df,columns,view=False):
        ret = df[columns].duplicated().sum()
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # ユニークデータ取得
    def get_uniq_data(df,uniq_key,sort_key,keep='first'):
        ret = df.sort_values(by=sort_key)
        ret.drop_duplicates(subset=uniq_key, keep=keep, inplace=True)
        return ret
    
    # カテゴリ集計
    def freq(df,columns,view=False):
        ret=list()
        for name in columns:
            tmp_df=df[name].value_counts()
            tmp_df.name=name
            #ret = pd.concat([ret,tmp_df],axis=1)
            ret.append(tmp_df)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            for r in ret:
                print(r)
                #display(r)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 複雑な集計
    def tabulate(df,row,col=None,var='',func=np.sum,view=False):
        if var == '':
            tmp_df=df.reset_index(drop=False,inplace=False)
            ret=pd.pivot_table(data=tmp_df, values='index', index=row, columns=col, aggfunc='count', dropna=False, fill_value=0 ,margins = False)
            tmp_df=None
        else:
            ret=pd.pivot_table(data=df, values=var, index=row, columns=col, aggfunc=func, dropna=False, fill_value=0 ,margins = False)
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # マージ
    def merge(df1,df2,key,how,view=True):
        if view:
            print("df1のキー重複")
            pandas_tool.check_dup_comb(df1,key,True)
            print("df2のキー重複")
            pandas_tool.check_dup_comb(df2,key,True)
            
            print("df1のオブザベーション:{0}".format(len(df1)))
            print("df2のオブザベーション:{0}".format(len(df2)))
        
        ret=pd.merge(df1,df2,how=how,on=key)
        
        if view:
            print("mergeのオブザベーション:{0}".format(len(ret)))
        
        return ret
    
    # Rank
    def rank(df,var,num,suffix='_rank',check=False):
        labels=[i for i in range(0,num)]
        df[var+suffix]=pd.qcut(df[var], num, labels=labels)
        
        # check data
        if check:
            ret=pd.DataFrame()
            max_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func=np.max,view=False)
            max_df.name='max'
            min_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func=np.min,view=False)
            min_df.name='min'
            cnt_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func='count',view=False)
            cnt_df.name='count'
            ret=pd.concat([ret,min_df,max_df,cnt_df],axis=1)
            return df,ret
            
        return df
    
    # Rank適用(min基準)
    def apply_rank(df,rank_df):
        tmp_df=copy.deepcopy(rank_df)
        tmp_df.reset_index(drop=False,inplace=True)
        target_name=tmp_df.columns[3]
        tmp_df.columns=["rank","min","max","cnt"]
        
        def judge_thld(row):
            ret_var = -1
            cond_list = ["if 0 : ret_var = 0"]
            
            for i in range(1,len(tmp_df)):
                cond_list.append("elif row < " +str(tmp_df.ix[i,'min'])+ " : ret_var = " + str(tmp_df.ix[i-1,'rank']))
            
            cond_list.append("else: ret_var = " + str(tmp_df.ix[len(tmp_df)-1,'rank']))
            cond_str="\r\n".join(cond_list)
            # ローカル辞書をexecと共有する
            local_dict=locals()
            exec(cond_str,local_dict)
            return local_dict["ret_var"]
        
        df[target_name+"_rank"]=df[target_name].apply(judge_thld)
        return df
    
    # Min%以下はMin%点に、Max%以上はMax%点にクリップする
    def clip_min_max(df,col_list,apply_df=None,max_pct=0.99,min_pct=0.01):
        p_min = df[col_list].quantile(min_pct)
        p_max = df[col_list].quantile(max_pct)
        
        df[col] = df[col_list].clip(p_min,p_max,axis=1)
        
        # もしも適用先のデータがあるならば（例えば検証データ）対応
        if apply_df is not None:
            apply_df[col] = apply_df[col_list].clip(p_min,p_max,axis=1)
            return df,apply_df
        else:
            return df
    
    
    # 文字列→数値変換
    def conv_float(df,column,percent_flg=False):
        
        def conv_f(row):
            if row[column] == "" or row[column] is np.nan:
                return np.nan
            else:
                return float(row[column])
        
        df[column]=df[column].str.replace("\\","").str.replace(",","").str.replace("%","").str.strip()
        df[column]=df.apply(conv_f,axis=1)
        
        if percent_flg:
            df[column]=df[column]/100
        
        return df

## 読み込み

In [3]:
#データ読み込み マークダウンにしてあるから必要ならコードセルにする
"""
data1を加工したcsvファイルを読み込む場合　
"""

name_csv = "crlea_bunya_dm_0613received_0613cleaned_filled_test.csv" #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#データが置いてあるフォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_csv)#ファイルパスとファイル名

# csvファイルの読み込みと空のリストに追加
df = pd.read_csv(filepath_or_buffer = path_file, sep=",",
                 usecols=['科目',
                          '分野名_修正v1',
                          '分野名_修正v2',
                          'ポイント採点',
                          '年度',
                          '採点回',
                          '学年',
                          '置換後のポイント数',
                          # '置換後の文字数',
                          '置換後の文字数5',
                          # '解答言語',
                          '置換後の配点',
                          '企画ペース',
                          '最終ペース'])
#列のリネーム
df = df.rename(columns={'分野名_修正v1':'分野1',
                        '分野名_修正v2':'分野2',
                        '置換後のポイント数':'ポイント数',
                        # '置換後の文字数':'文字数',#英語1単語も1文字として数えた
                        '置換後の文字数5':'文字数',#英語１単語を5文字とした。前の分析ではこっち。精度を比較する。
                        '置換後の配点':'配点'})
df_raw = df

df1 = df_raw
df1 = df1.dropna(subset=['ポイント数', '配点', '文字数'])
df1.isnull().sum()

科目          0
企画ペース     171
最終ペース       0
年度          0
採点回         0
学年          0
分野1         0
分野2         0
ポイント採点      0
文字数         0
ポイント数       0
配点          0
dtype: int64

### DecisionTreeRegressorメソッドの概要

|引数名|概要|デフォルト|
| :---- | :---- | :---- |
|criterion|不純度を測定する基準（平均二乗誤差、平均絶対誤差など）|‘mse’|
|splitter|条件探索アルゴリズムを選択するオプション（’best’と’rondom’が指定可能）|‘best’|
|max_depth|決定木のノード深さの制限値。ツリーが深くなりすぎて過学習の状態に陥った際は、このパラメータが正則化の役割を果たす。|None|
|min_samples_split|ノードを分割するために必要なサンプルの最小値|2|
|min_samples_leaf|1ノードの深さを作成するために必要となるデータ数の最小値。指定した値以上のデータ数を持たないノードは作られない。|1|
|min_weight_fraction_leaf|サンプルの重みを考慮した上でのmin_samples_leafに該当|0.0|
|max_features|ランダムに指定する説明変数の数(全ての説明変数がモデル学習に活用されるわけではなく、ランダムに割り振られる）|None|
|random_state|乱数シード|None|
|max_leaf_nodes|作成される決定木の葉の数を、指定した値以下に制御する|None|
|min_impurity_decrease|決定木の成長の早期停止するための閾値。不純度が指定の値より減少した場合、ノードを分岐し、不純度が指定の値より減少しなければ分岐を抑制。|0.0|
|ccp_alpha|ccp_alphaが大きいほどプルーニングされるノードの数が増加。プルーニングとは、精度低下をできるだけ抑えながら過剰な重みを排除するプロセスを指す。|0.0|


### 関数の定義

In [1]:
"""
作業方針
20年と２１年で分ける
２０年の分野別最終ペース平均を作る
２１年にマージする
２１年の夏とそれ以外で機械学習を実施
結果を検証する
"""

'\n作業方針\n20年と２１年で分ける\n２０年の分野別最終ペース平均を作る\n２１年にマージする\n２１年の夏とそれ以外で機械学習を実施\n結果を検証する\n'

In [28]:
def te_20to21_order(df):
    df = df.dropna(subset=['ポイント数', '配点', '文字数'])
    df.loc[:,'ポイント数'] =df.loc[:,'ポイント数'].round(0).astype(int)
    df.loc[:,'配点'] = df.loc[:,'配点'].round(0).astype(int)
    df.loc[:,'文字数'] = df.loc[:,'文字数'].round(0).astype(int)
    df = df[df['科目']=="数学"]

    #データフレームの分離
    df1 = df[df['年度']==20] #20年のデータ。target encording
    df2 = df[df['年度']==21] #21年のデータ。機械学習用

    bunya1 = df1.groupby('分野1', as_index=False)['最終ペース'].mean()
    bunya1= bunya1.rename(columns = {"最終ペース":"分野1TS"})
    bunya1=bunya1.sort_values('分野1TS',ascending=True).reset_index()
    order =pd.Series(data = [1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8])
    bunya1['分野1_order']=order

    df2 = pandas_tool.merge(df2,bunya1, how='left', key='分野1')
    
    df2 = df2.drop(columns = ['index','分野1TS','分野2'])

    #機械学習の訓練と検証
    """秋冬が訓練用"""
    #訓練用説明変数
    rem_cols= ["企画ペース","採点回","年度"]
    train = df2[~(df2['採点回'] == "夏")]
    train = train.drop(columns = rem_cols)
    
    X_train =train.drop('最終ペース', axis=1)
    y_train =train['最終ペース']

    """夏が検証用"""
    #訓練用説明変数
    test = df2[(df2['採点回'] == "夏")]
    X_test =test.drop(columns=['最終ペース',"企画ペース","採点回","年度"])
    y_test =test['最終ペース']
    y_kikaku =test['企画ペース']
    y_kamoku = test['科目']

    df_res=[X_train, y_train, X_test,  y_test, y_kikaku, y_kamoku]#学年混合モデルデータ

    return df_res

In [26]:
#データ数のチェック
df2 = te_20to21_order(df_raw)
print("X_train" + str(len(df2[0])))
print("X_testは" + str(len(df2[1])))
print("y_trainは" + str(len(df2[2])))
print("y_testは" + str(len(df2[3])))

df1のキー重複
90
df2のキー重複
0
df1のオブザベーション:106
df2のオブザベーション:16
mergeのオブザベーション:106
X_train66
X_testは66
y_trainは40
y_testは40


In [27]:
df2[0].head(5)

,科目,学年,分野1,ポイント採点,文字数,ポイント数,配点,分野1_order
0,数学,1,数と式,0.0,3,10,10,4
1,数学,1,場合の数と確率,0.0,3,10,20,6
2,数学,1,整数の性質,0.0,3,10,20,3
3,数学,1,図形の性質,0.0,3,10,20,4
4,数学,2,場合の数と確率,0.0,3,10,40,6


In [30]:
"""
make_mldataのseriesを使って、予測値を出し、予測値、最終ペース、差分、乖離度（予測値/最終ペース）を列に追加したdfを返す。
"""
def test_model(df,a=2,b=1,c=None):
    X_train= df[0]
    y_train= df[1]
    X_test = df[2]
    y_test = df[3]
    kikaku = df[4]
    kamoku = df[5]
    rem_cols= ["科目","分野1"]
    X_train = X_train.drop(columns = rem_cols)
    X_test = X_test.drop(columns = rem_cols)
    
    X_test_excel = df[2]
    rem_cols= ["科目"]
    X_test_excel = X_test_excel.drop(columns = rem_cols)
    
    
    
    
    model = DecisionTreeRegressor(criterion='mse', 
                                   splitter='best', 
                                   max_depth=c, 
                                   min_samples_split=a, #3,4,5とか？
                                   min_samples_leaf=b,#2とか 
                                   min_weight_fraction_leaf=0.0,
                                   max_features=None, 
                                   random_state=None, 
                                   max_leaf_nodes=None, 
                                   min_impurity_decrease=0.0, 
                                   ccp_alpha=0.0
                                  )

    #上記のパラメータでモデルを学習する
    model.fit(X_train, y_train)
    y_pred  = model.predict(X_test)
    #得た結果の樹形図を表示する
    
    #得た結果を学習データとマージしてデータフレームで返す
    df_res=[]
    
    #検証に使った説明変数データのダミー変数を元に戻す
    df_res = X_test_excel 
    df_res.loc[:,'AI想定ペース']= y_pred #上のデータに予測値をマージ
    df_res.loc[:,'最終ペース']= y_test
    df_res.loc[:,'企画ペース']= kikaku
    df_res.loc[:,'科目']= kamoku
    df_res.loc[:,'誤差']= df_res['AI想定ペース']-df_res['最終ペース']#実測値と予測値の差分を列に追加 
    df_res.loc[:,'AI乖離度']= df_res['AI想定ペース']/df_res['最終ペース']#実測値と予測値の差分を列に追加 
    df_res.loc[:,'元の乖離度']= df_res['企画ペース']/df_res['最終ペース']#実測値と予測値の差分を列に追加 
            
    # 最終ペースをランク分け
    x = "最終ペース"
    conditions = [
        (df_res[x] >= 600),
        (df_res[x] >= 500),
        (df_res[x] >= 400),
        (df_res[x] >= 300),
        (df_res[x] >= 200),
        (df_res[x] >= 100),
        (df_res[x] >= 0)
         ]

    choices = ["600~", "500~600", "400~500", "300~400", "200~300", "100~200","0~100"]

    df_res.loc[:,'最終ランク'] = np.select(conditions, choices, default = 0)
    return df_res

In [29]:
def make_leaf(df,a=2,b=1,c=None):
    X_train= df[0]
    y_train= df[1]
    X_test = df[2]
    y_test = df[3]
    rem_cols= ["科目","分野1"]
    X_train = X_train.drop(columns = rem_cols)
    X_test = X_test.drop(columns = rem_cols)
    X_train_feature_names = X_train.columns.values.tolist() 
  
    model = DecisionTreeRegressor(criterion='mse', 
                                   splitter='best', 
                                   max_depth=c, 
                                   min_samples_split=a, #3,4,5とか？
                                   min_samples_leaf=b,#2とか 
                                   min_weight_fraction_leaf=0.0,
                                   max_features=None, 
                                   random_state=None, 
                                   max_leaf_nodes=None, 
                                   min_impurity_decrease=0.0, 
                                   ccp_alpha=0.0
                                  )

    #上記のパラメータでモデルを学習する
    model.fit(X_train, y_train)
    dot_data = tree.export_graphviz(model, out_file=None,
                                feature_names= X_train_feature_names,
                                # class_names=iris.target_names,
                                filled=True, rounded=True, special_characters=True
                               ) 
    graph = graphviz.Source(dot_data) 
    return graph

# 結果出力

In [31]:
"""
Excelの書き出し 
"""

df_v = test_model(te_20to21_order(df_raw))
df_sum = pandas_tool.summary(df_v.loc[:,['最終ペース','企画ペース','AI想定ペース','誤差','AI乖離度','元の乖離度']])[0]


name_excel_output = "分野別ペース順序変数8_20target_21機械学習_分野1TS.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/output"#Excelが置いてあるフォルダパス


with pd.ExcelWriter('{}/{}'.format(path_folder,name_excel_output)) as writer:
    df_v.to_excel(writer, sheet_name='value',encoding='utf-8-sig', index = False)
    df_sum.to_excel(writer, sheet_name='summary',encoding='utf-8-sig', index = True)

df1のキー重複
90
df2のキー重複
0
df1のオブザベーション:106
df2のオブザベーション:16
mergeのオブザベーション:106


In [34]:
df = te_20to21_order(df_raw)
fig = make_leaf(df)

name_file = "分野別ペース順序変数8_20target_21機械学習_分野1TS" #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/output"#フォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_file)#ファイルパスとファイル名
fig.render(path_file)

df1のキー重複
90
df2のキー重複
0
df1のオブザベーション:106
df2のオブザベーション:16
mergeのオブザベーション:106


'/Users/s.ogura/Documents/CRLEA/data/output/分野別ペース順序変数8_20target_21機械学習_分野1TS.pdf'